# Selecting a Model

## Import Necessary Packages

In [15]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import re
import pickle
from itertools import combinations

## Data Cleaning

In [16]:
df = pd.read_csv('../data/cleaned_car_price_prediction_door_fix.csv')
df['HasTurbo'] = df['HasTurbo'].astype(int)
df.columns = [re.sub(' ', '_', col) for col in df.columns]
df.columns = [re.sub('\.', '', col) for col in df.columns]
x_cols = df.columns[1:]
df = df.sample(n=2000, random_state=42) # select a subset
df.head()
df.Doors.value_counts()

Doors
4     1913
2       81
>5       6
Name: count, dtype: int64

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2000 entries, 736 to 15704
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Price             2000 non-null   int64  
 1   Manufacturer      2000 non-null   object 
 2   Prod_year         2000 non-null   int64  
 3   Category          2000 non-null   object 
 4   Leather_interior  2000 non-null   object 
 5   Fuel_type         2000 non-null   object 
 6   Engine_Volume     2000 non-null   float64
 7   HasTurbo          2000 non-null   int64  
 8   Mileage           2000 non-null   int64  
 9   Cylinders         2000 non-null   float64
 10  Gear_box_type     2000 non-null   object 
 11  Drive_wheels      2000 non-null   object 
 12  Doors             2000 non-null   object 
 13  Wheel             2000 non-null   object 
 14  Color             2000 non-null   object 
 15  Airbags           2000 non-null   int64  
dtypes: float64(2), int64(5), object(9)
memory us

## Fitting full MLR model

In [18]:
formula = "Price ~ " #initiate the formula
for i, predictor in enumerate(x_cols):
    if i == 0:
        if df.dtypes[predictor] == 'object': 
            formula += f'C({predictor})'
        else:
            formula += predictor
    else:
        if df.dtypes[predictor] == 'object':
            formula += f' + C({predictor})'
        else:
            formula += f' + {predictor}'
full_model = smf.ols(formula, data = df).fit()
print(full_model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.354
Model:                            OLS   Adj. R-squared:                  0.323
Method:                 Least Squares   F-statistic:                     11.74
Date:                Fri, 11 Oct 2024   Prob (F-statistic):          8.21e-125
Time:                        15:28:23   Log-Likelihood:                -22046.
No. Observations:                2000   AIC:                         4.427e+04
Df Residuals:                    1910   BIC:                         4.478e+04
Df Model:                          89                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercep

## Funtion Creations

### Generate every combination of all subsets

In [19]:
def all_subsets(lst):
    subsets = []
    # Iterate over all possible lengths of the subset
    for r in range(len(lst) + 1):
        # Generate all combinations of length r
        for combo in combinations(lst, r):
            subsets.append(list(combo))
    return subsets

### Calculate some metrics given a model, predictors, and response

In [20]:
#calculate some metrics given a model, predictors, and response
def calculate_metrics(model, X, y):
    n = len(y)
    k = model.df_model  # Number of predictors, excluding intercept
    
    # AIC
    aic = model.aic
    
    # BIC
    bic = model.bic
    
    # PRESS (Prediction Sum of Squares)
    hat_matrix = X @ np.array(np.linalg.inv(X.T @ X) @ X.T)
    residuals = model.resid
    press = np.sum((residuals / (1 - np.diag(hat_matrix)))**2)
        
    # Adjusted R-squared
    r2 = model.rsquared
    adj_r2 = 1 - (1 - r2) * (n - 1) / (n - k - 1)
    
    # Mean Squared Error MSE
    residuals = model.resid
    mse = (residuals ** 2).mean()
    
    return aic, bic, press, adj_r2, int(k), mse #dont consider intercept as a predictor

### Perform Cross Validation and find the MSE

In [21]:
def cross_validate(X, y, k=5):
    np.random.seed(42)  # For reproducibility
    shuffled_indices = np.random.permutation(len(X))
    fold_sizes = len(X) // k
    scores = []
    for i in range(k):
        val_start = i * fold_sizes
        val_end = val_start + fold_sizes
        val_indices = shuffled_indices[val_start:val_end]
        train_indices = np.concatenate([shuffled_indices[:val_start], shuffled_indices[val_end:]])
        # print(val_indices, train_indices)

        X_train, y_train = X.iloc[train_indices], y.iloc[train_indices]
        X_val, y_val = X.iloc[val_indices], y.iloc[val_indices]
        
        #calculate mse if you train the model with the training set
        model = sm.OLS(y_train, X_train).fit()
        predictions = model.predict(X_val)
        mse = ((predictions - y_val.squeeze()) ** 2).mean()
        scores.append(mse)

    return sum(scores) / len(scores)

### Calculate results and save to pickle file

In [22]:
def results_to_pkl(subsets, index = ''):     
    results = []
    for j, predictors in enumerate(subsets):
        # if j == 7: break
        # print(predictors)
        formula = "Price ~ " #initiate the formula
        for i, predictor in enumerate(predictors):
            if i == 0:
                if df.dtypes[predictor] == 'object':
                    formula += f'C({predictor})'
                else:
                    formula += predictor
            else:
                if df.dtypes[predictor] == 'object':
                    formula += f' + C({predictor})'
                else:
                    formula += f' + {predictor}'
                    
        #train the model            
        model = smf.ols(formula, data = df).fit()
        
        #set up X and y for calculate_metrics adn cross_validate 
        y = df['Price']
        X = df[predictors].copy()
        cat_cols = []
        for predictor in predictors: 
            if X.dtypes[predictor] == 'object':
                cat_cols.append(predictor)
        if len(cat_cols) > 0:
            X = pd.get_dummies(X, columns=cat_cols, drop_first=True)
            for col in X.columns:
                X[col] = X[col].astype(float)
        X = sm.add_constant(X, has_constant='add')
                
        aic, bic, press, adj_r2, num_predictors, mse = calculate_metrics(model, X = X, y = y)
        mse_cv_5 = cross_validate(X, y, k = 5)
        mse_cv_10 = cross_validate(X, y, k = 10)
        # mse_cv_100 = cross_validate(X, y, k = 100)
        
        results.append({
            'Predictors': predictors,
            'n_Predictors': num_predictors,
            'Adjusted R^2': adj_r2,
            'AIC': aic,
            'BIC': bic,
            'PRESS': press,
            'MSE': mse,
            '5-Fold_CV MSE': mse_cv_5,
            '10-Fold_CV MSE': mse_cv_10,
            # '100-Fold_CV MSE': mse_cv_100        
        })
            
    # Convert results to pd DataFrame
    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values(by='n_Predictors').reset_index(drop=True)
    with open(f'models/results_df_{index}.pkl', 'wb') as f:
        pickle.dump(results_df, f)

## Testing Functions on Full Model

In [23]:
results_to_pkl([x_cols], index = 'full_model')
with open(f'models/results_df_full_model.pkl', 'rb') as f:
    data = pickle.load(f)
data

,Predictors,n_Predictors,Adjusted R^2,AIC,BIC,PRESS,MSE,5-Fold_CV MSE,10-Fold_CV MSE
0,"Index(['Manufacturer', 'Prod_year', 'Category'...",89,0.323434,44271.767731,44775.848952,1.177482e+24,2.197513e+08,1.585317e+10,1.662179e+10


## Finding the Best Subset

### Creating all subsets and sectioning them off 

In [24]:
subsets = all_subsets(x_cols) #list of all predictors
subsets = subsets[1:]
len(subsets)

32767

In [25]:
n = 100
k = len(subsets) // n 
sections = []
for i in range(n):
    if i == n - 1:
        sections += [subsets[i*k:]]
    else: 
        sections += [subsets[i*k:i*k+k]]
sum([len(section) for section in sections])
subsets[k:k+10] == sections[1][:10]

True

### Testing on the random index

In [26]:
results_to_pkl(sections[47], 47)

In [27]:
with open(f'models/results_df_47.pkl', 'rb') as f:
    data = pickle.load(f)
data

,Predictors,n_Predictors,Adjusted R^2,AIC,BIC,PRESS,MSE,5-Fold_CV MSE,10-Fold_CV MSE
0,"[Leather_interior, Fuel_type, Engine_Volume, M...",11,0.086021,44797.374243,44864.585073,9.560712e+13,3.089871e+08,3.902114e+10,4.370982e+10
1,"[Leather_interior, Fuel_type, Engine_Volume, H...",11,0.119269,44723.263596,44790.474425,1.050376e+14,2.977470e+08,4.015779e+10,4.533714e+10
2,"[Leather_interior, Fuel_type, Engine_Volume, H...",11,0.111727,44740.317467,44807.528296,9.279032e+13,3.002967e+08,3.482777e+10,4.007582e+10
3,"[Leather_interior, Fuel_type, Engine_Volume, H...",11,0.114739,44733.525466,44800.736296,1.070141e+14,2.992787e+08,4.103108e+10,4.596733e+10
4,"[Leather_interior, Fuel_type, Engine_Volume, H...",11,0.119451,44722.849979,44790.060809,6.032950e+11,2.976854e+08,3.008719e+08,3.011131e+08
...,...,...,...,...,...,...,...,...,...
322,"[Category, Fuel_type, Cylinders, Gear_box_type...",35,0.159527,44653.397492,44855.029981,5.785253e+11,2.807069e+08,2.892778e+08,2.898945e+08
323,"[Category, Fuel_type, Gear_box_type, Doors, Wh...",36,0.158424,44657.002023,44864.235414,5.782512e+11,2.809322e+08,2.893221e+08,2.900816e+08
324,"[Category, Fuel_type, Gear_box_type, Drive_whe...",36,0.162862,44646.427894,44853.661285,5.758543e+11,2.794508e+08,2.878597e+08,2.884234e+08
325,"[Category, Fuel_type, Gear_box_type, Drive_whe...",37,0.148632,44681.119760,44893.954054,5.861262e+11,2.840562e+08,2.938311e+08,2.940034e+08


### Finding the results for all the subsets

In [28]:
for i in range(n):
    results_to_pkl(sections[i], i)

KeyboardInterrupt: 

In [15]:
dataframes = []
for i in range(n):
    with open(f'models/results_df_{i}.pkl', 'rb') as f:
        data = pickle.load(f)
    #concat all the data into one dataframe
    dataframes.append(data)
results_df = pd.concat(dataframes, ignore_index=True)
results_df.to_csv('models/all_results_df.csv', index=False)
results_df

,Predictors,n_Predictors,Adjusted R^2,AIC,BIC,PRESS,MSE,5-Fold_CV MSE,10-Fold_CV MSE
0,[Wheel],1,0.017426,44932.145767,44943.347572,6.684511e+11,3.338478e+08,3.341604e+08,3.340392e+08
1,[Prod_year],1,0.085955,44787.554836,44798.756641,6.231931e+11,3.105639e+08,3.110198e+08,3.112046e+08
2,[Leather_interior],1,0.016570,44933.888300,44945.090105,6.693437e+11,3.341388e+08,3.347360e+08,3.345047e+08
3,[Engine_Volume],1,0.011283,44944.611106,44955.812911,6.741469e+11,3.359351e+08,3.367175e+08,3.366080e+08
4,[HasTurbo],1,0.046945,44871.139192,44882.340997,6.495586e+11,3.238181e+08,3.242530e+08,3.242776e+08
...,...,...,...,...,...,...,...,...,...
32762,"[Manufacturer, Prod_year, Category, Leather_in...",88,0.307411,44317.628274,44816.108593,2.329468e+20,2.250734e+08,1.384365e+10,1.455920e+10
32763,"[Manufacturer, Prod_year, Category, Leather_in...",88,0.322453,44273.712290,44772.192609,6.356887e+20,2.201851e+08,1.385893e+10,1.476848e+10
32764,"[Manufacturer, Prod_year, Category, Fuel_type,...",88,0.323788,44269.769083,44768.249402,1.410980e+20,2.197514e+08,1.578653e+10,1.653652e+10
32765,"[Manufacturer, Category, Leather_interior, Fue...",88,0.234284,44518.378779,45016.859097,inf,2.488379e+08,3.771471e+10,3.949092e+10


## Analysis

In [16]:
print('Lowest MSE')
results_df.sort_values('MSE').head(5)

Lowest MSE


,Predictors,n_Predictors,Adjusted R^2,AIC,BIC,PRESS,MSE,5-Fold_CV MSE,10-Fold_CV MSE
32766,"[Manufacturer, Prod_year, Category, Leather_in...",89,0.323434,44271.767731,44775.848952,1.364124e+21,2.197513e+08,1.585317e+10,1.662179e+10
32764,"[Manufacturer, Prod_year, Category, Fuel_type,...",88,0.323788,44269.769083,44768.249402,1.410980e+20,2.197514e+08,1.578653e+10,1.653652e+10
32761,"[Manufacturer, Prod_year, Category, Leather_in...",88,0.323777,44269.801112,44768.281431,1.524720e+14,2.197549e+08,2.490515e+08,2.466119e+08
32754,"[Manufacturer, Prod_year, Category, Fuel_type,...",87,0.324130,44267.802051,44760.681468,5.135175e+14,2.197550e+08,2.489379e+08,2.464479e+08
32758,"[Manufacturer, Prod_year, Category, Leather_in...",88,0.323725,44269.954882,44768.435201,8.451237e+22,2.197718e+08,1.584767e+10,1.661215e+10


In [17]:
print('Lowest 5 fold CV MSE')
results_df.sort_values('5-Fold_CV MSE').head(5)

Lowest 5 fold CV MSE


,Predictors,n_Predictors,Adjusted R^2,AIC,BIC,PRESS,MSE,5-Fold_CV MSE,10-Fold_CV MSE
28559,"[Manufacturer, Prod_year, Category, Fuel_type,...",69,0.324804,44248.547644,44640.610816,3.617383e+13,2.216027e+08,2.460270e+08,2.444512e+08
23305,"[Manufacturer, Prod_year, Category, Fuel_type,...",67,0.323574,44250.260426,44631.121794,5.608619e+15,2.222366e+08,2.461083e+08,2.445209e+08
23325,"[Manufacturer, Prod_year, Category, Fuel_type,...",68,0.324988,44247.039093,44633.501363,1.211362e+13,2.216572e+08,2.461122e+08,2.444605e+08
30879,"[Manufacturer, Prod_year, Category, Leather_in...",70,0.324470,44250.500728,44648.164802,2.847906e+12,2.215975e+08,2.461723e+08,2.445781e+08
16806,"[Manufacturer, Prod_year, Category, Fuel_type,...",66,0.323742,44248.798109,44624.058574,3.305393e+13,2.222964e+08,2.462039e+08,2.445568e+08


In [18]:
print('Lowest 10 fold CV MSE')
results_df.sort_values('10-Fold_CV MSE').head(5)

Lowest 10 fold CV MSE


,Predictors,n_Predictors,Adjusted R^2,AIC,BIC,PRESS,MSE,5-Fold_CV MSE,10-Fold_CV MSE
28559,"[Manufacturer, Prod_year, Category, Fuel_type,...",69,0.324804,44248.547644,44640.610816,3.617383e+13,2.216027e+08,2.460270e+08,2.444512e+08
23325,"[Manufacturer, Prod_year, Category, Fuel_type,...",68,0.324988,44247.039093,44633.501363,1.211362e+13,2.216572e+08,2.461122e+08,2.444605e+08
28221,"[Manufacturer, Prod_year, Category, Fuel_type,...",69,0.326280,44244.169679,44636.232852,5.709619e+11,2.211182e+08,2.462582e+08,2.444868e+08
23305,"[Manufacturer, Prod_year, Category, Fuel_type,...",67,0.323574,44250.260426,44631.121794,5.608619e+15,2.222366e+08,2.461083e+08,2.445209e+08
31137,"[Manufacturer, Prod_year, Category, Fuel_type,...",70,0.326012,44245.928356,44643.592430,1.027410e+12,2.210915e+08,2.462350e+08,2.445535e+08


In [19]:
print('Lowest AIC')
results_df.sort_values('AIC').head(5)

Lowest AIC


,Predictors,n_Predictors,Adjusted R^2,AIC,BIC,PRESS,MSE,5-Fold_CV MSE,10-Fold_CV MSE
28221,"[Manufacturer, Prod_year, Category, Fuel_type,...",69,0.326280,44244.169679,44636.232852,5.709619e+11,2.211182e+08,2.462582e+08,2.444868e+08
31137,"[Manufacturer, Prod_year, Category, Fuel_type,...",70,0.326012,44245.928356,44643.592430,1.027410e+12,2.210915e+08,2.462350e+08,2.445535e+08
31142,"[Manufacturer, Prod_year, Category, Fuel_type,...",70,0.325936,44246.155729,44643.819804,1.666442e+16,2.211166e+08,1.472538e+10,1.612762e+10
30886,"[Manufacturer, Prod_year, Category, Leather_in...",70,0.325933,44246.164280,44643.828354,2.336561e+13,2.211176e+08,2.463565e+08,2.446656e+08
23325,"[Manufacturer, Prod_year, Category, Fuel_type,...",68,0.324988,44247.039093,44633.501363,1.211362e+13,2.216572e+08,2.461122e+08,2.444605e+08


In [20]:
print('Lowest BIC')
results_df.sort_values('BIC').head(5)

Lowest BIC


,Predictors,n_Predictors,Adjusted R^2,AIC,BIC,PRESS,MSE,5-Fold_CV MSE,10-Fold_CV MSE
7694,"[Prod_year, Fuel_type, Engine_Volume, HasTurbo...",12,0.254162,44391.771257,44464.582989,5.128801e+11,2.520174e+08,2.544981e+08,2.557167e+08
14161,"[Prod_year, Fuel_type, Engine_Volume, HasTurbo...",13,0.255234,44389.886018,44468.298652,5.121325e+11,2.515283e+08,2.541019e+08,2.555276e+08
7980,"[Prod_year, Fuel_type, HasTurbo, Cylinders, Ge...",12,0.252567,44396.041549,44468.853281,5.140421e+11,2.525560e+08,2.551993e+08,2.563399e+08
14158,"[Prod_year, Fuel_type, Engine_Volume, HasTurbo...",13,0.254915,44390.742400,44469.155034,5.135274e+11,2.516360e+08,2.549607e+08,2.558011e+08
14162,"[Prod_year, Fuel_type, HasTurbo, Cylinders, Ge...",13,0.254441,44392.015175,44470.427809,5.127143e+11,2.517962e+08,2.545187e+08,2.558312e+08


In [21]:
print('Highest Adjusted R^2	')
results_df.sort_values('Adjusted R^2', ascending=False).head(5)

Highest Adjusted R^2	


,Predictors,n_Predictors,Adjusted R^2,AIC,BIC,PRESS,MSE,5-Fold_CV MSE,10-Fold_CV MSE
28221,"[Manufacturer, Prod_year, Category, Fuel_type,...",69,0.326280,44244.169679,44636.232852,5.709619e+11,2.211182e+08,2.462582e+08,2.444868e+08
31137,"[Manufacturer, Prod_year, Category, Fuel_type,...",70,0.326012,44245.928356,44643.592430,1.027410e+12,2.210915e+08,2.462350e+08,2.445535e+08
31142,"[Manufacturer, Prod_year, Category, Fuel_type,...",70,0.325936,44246.155729,44643.819804,1.666442e+16,2.211166e+08,1.472538e+10,1.612762e+10
30886,"[Manufacturer, Prod_year, Category, Leather_in...",70,0.325933,44246.164280,44643.828354,2.336561e+13,2.211176e+08,2.463565e+08,2.446656e+08
31143,"[Manufacturer, Prod_year, Category, Fuel_type,...",71,0.325734,44247.718414,44650.983391,4.302740e+14,2.210683e+08,2.469552e+08,2.450436e+08


In [22]:
print('Lowest Press')
results_df.sort_values('PRESS').head(5)

Lowest Press


,Predictors,n_Predictors,Adjusted R^2,AIC,BIC,PRESS,MSE,5-Fold_CV MSE,10-Fold_CV MSE
28914,"[Manufacturer, Prod_year, Leather_interior, Ha...",71,0.288349,44355.644436,44758.909413,5.003152e+11,2.333255e+08,2.573204e+08,2.554305e+08
10320,"[Manufacturer, Prod_year, Category, Fuel_type,...",65,0.308728,44291.749380,44661.408942,5.007999e+11,2.273492e+08,2.521758e+08,2.498940e+08
28868,"[Manufacturer, Prod_year, Leather_interior, Fu...",62,0.308613,44289.180565,44642.037420,5.012719e+11,2.277396e+08,2.497033e+08,2.482216e+08
26248,"[Prod_year, Category, Fuel_type, HasTurbo, Cyl...",24,0.271717,44356.016834,44496.039395,5.035307e+11,2.445992e+08,2.497604e+08,2.515128e+08
26259,"[Prod_year, Category, Fuel_type, Engine_Volume...",24,0.271727,44355.989279,44496.011840,5.036665e+11,2.445958e+08,2.498719e+08,2.515708e+08
